In [1]:
import concurrent.futures
import requests
import json
import random

In [2]:
import psycopg2

In [3]:
from config import config, cross

In [4]:
def verify_nft(id_sent):
    cross_params = cross()
    headers = {
        'x-client-secret': cross_params['cross_secret'],
        'x-project-id': cross_params['cross_id'],
    }
    response = requests.get('https://staging.crossmint.io/api/2022-06-09/collections/default/nfts/' + id_sent, headers=headers, timeout=10)
    return response.json()

In [5]:
def get_tx_to_monitor(id_batchs):
    """
    A dataframe from aggregated_investments table in order to save a snapshot
    Returns:
        snapshot: a dataframe with all the relevan fields to save
    """
    sql = "SELECT id_sent FROM sent WHERE batch = %s"
    conn = None
    data = []
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(sql, (id_batchs,))
        email_users = cur.fetchall()
        for row in email_users:
            data.append({'id_sent': row[0]})
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    return data

In [6]:
id_batchs = 4
transactions = get_tx_to_monitor(id_batchs = id_batchs)

In [7]:
transactions

[{'id_sent': '29b16ad4-8fc0-499c-a229-9b7363d2dae9'},
 {'id_sent': 'c03dcc4a-f1c8-452f-9bbe-3f76697a0ee8'}]

In [11]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for id_operation in transactions:
        futures.append(executor.submit(verify_nft, id_sent = id_operation['id_sent']))
    for future in concurrent.futures.as_completed(futures):
        print(future.result())

{'id': '29b16ad4-8fc0-499c-a229-9b7363d2dae9', 'onChain': {'status': 'pending', 'chain': 'solana'}}
{'id': 'c03dcc4a-f1c8-452f-9bbe-3f76697a0ee8', 'onChain': {'status': 'pending', 'chain': 'solana'}}


In [ ]:
# falta hacer que guarde los txid en el banco en la tabla sent y que evite que de time out.